# Modelos Ensamble

Vamos a probar para nuestro modelos distintos modelos de ensamble a ver cual es el que mejor funciona.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("../")
from src import funciones_problemas_regresion as fpr

In [2]:
df =  pd.read_csv("../datos/df_autorenew_nonulls_estand_sinout_encoded.csv", index_col=0).reset_index(drop=True)
df_sample = df.sample(30000)
variable_respuesta = "price"

________________________________________

## Random Forest Regressor

In [3]:
variable_respuesta = "price"
modelo = "RandomForestRegressor"

# Inicializamos la clase, separamos VR y VD y separamos el train y el test.
clase_arbol = fpr.ProblemaRegresion(df_sample, variable_respuesta, modelo)
clase_arbol.separar_variables()
clase_arbol.separar_train_test()

# Estudiamos cuales son los mejores parámetros, entrenamos al modelo y vemos las métricas.
params_rf = {
    'n_estimators': [50, 100, 150], #número de árboles que queremos en el bosque
    'max_depth': [15, 20, 30],
    'min_samples_split': [15, 20, 25],
    'min_samples_leaf': [15, 20, 25], 
    'max_leaf_nodes': [100, 200, 300]
}

clase_arbol.grid_search(params_rf, cv=3)

Calculando el modelo con RandomForestRegressor...
Las mejores métricas para el modelo de DecisionTreeRegressor son:
{'max_depth': 20, 'max_leaf_nodes': 300, 'min_samples_leaf': 15, 'min_samples_split': 15, 'n_estimators': 100}


,r2_scores,MAE,MSE,RMSE
train,0.784689,1788.570378,9.370644e+06,3061.150831
test,0.744358,1990.782470,1.184837e+07,3442.145686


In [4]:
params_rf_2 = {
    'n_estimators': [75, 100, 125], #número de árboles que queremos en el bosque
    'max_depth': [15, 20, 25],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [10, 15, 20],
    'max_leaf_nodes': [300, 400, 500]
}

clase_arbol.grid_search(params_rf_2, cv=3)

Calculando el modelo con RandomForestRegressor...
Las mejores métricas para el modelo de DecisionTreeRegressor son:
{'max_depth': 15, 'max_leaf_nodes': 500, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 125}


,r2_scores,MAE,MSE,RMSE
train,0.815577,1685.095612,8.294877e+06,2880.082770
test,0.732799,1919.721471,1.164419e+07,3412.357888


_____________________________________

## Gradient Boosting Regressor

In [4]:
modelo = "GradientBoostingRegressor"

# Inicializamos la clase, separamos VR y VD y separamos el train y el test.
clase_arbol = fpr.ProblemaRegresion(df_sample, variable_respuesta, modelo)
clase_arbol.separar_variables()
clase_arbol.separar_train_test()

# Estudiamos cuales son los mejores parámetros, entrenamos al modelo y vemos las métricas.
params_rf = {
    'n_estimators': [75, 100, 125], #número de árboles que queremos en el bosque
    'max_depth': [10, 15, 20],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [10, 15, 20],
    'max_leaf_nodes': [300, 400, 500]
}

clase_arbol.grid_search(params_rf, cv=3)

Calculando el modelo con GradientBoostingRegressor...
Las mejores métricas para el modelo de DecisionTreeRegressor son:
{'max_depth': 10, 'max_leaf_nodes': 400, 'min_samples_leaf': 15, 'min_samples_split': 10, 'n_estimators': 125}


,r2_scores,MAE,MSE,RMSE
train,0.895748,1336.855012,4.537209e+06,2130.072531
test,0.787655,1810.078595,9.841666e+06,3137.142938


## XGB Regressor

In [7]:
modelo = "XGBRegressor"

# Inicializamos la clase, separamos VR y VD y separamos el train y el test.
clase_arbol = fpr.ProblemaRegresion(df_sample, variable_respuesta, modelo)
clase_arbol.separar_variables()
clase_arbol.separar_train_test()

# Estudiamos cuales son los mejores parámetros, entrenamos al modelo y vemos las métricas.
params_xgb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 4],  # Equivalente a min_samples_leaf
    'subsample': [0.8, 1.0],        # Para evitar sobreajuste
    'colsample_bytree': [0.8, 1.0]  # Selección de columnas al construir cada árbol
}

clase_arbol.grid_search(params_xgb, cv=3)

Calculando el modelo con XGBRegressor...
Las mejores métricas para el modelo de DecisionTreeRegressor son:
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 150, 'subsample': 1.0}


,r2_scores,MAE,MSE,RMSE
train,0.865319,1559.126386,6.057622e+06,2461.223721
test,0.763334,1852.755420,1.031355e+07,3211.471231


In [8]:
def sacar_metricas(y_train, y_test, y_pred_train, y_pred_test):
    metricas = {
        'train': {
            'r2': r2_score(y_train, y_pred_train),
            'MAE': mean_absolute_error(y_train, y_pred_train),
            'MSE': mean_squared_error(y_train, y_pred_train),
            'RMSE': np.sqrt(mean_squared_error(y_train, y_pred_train))
        },
        'test': {
            'r2': r2_score(y_test, y_pred_test),
            'MAE': mean_absolute_error(y_test, y_pred_test),
            'MSE': mean_squared_error(y_test, y_pred_test),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_test))

        }
    }


    return  pd.DataFrame(metricas).T

In [10]:
from xgboost import XGBRegressor

X = df_sample.drop("price", axis = 1)
y = df_sample[["price"]]

X_train, X_test, y_train,y_test = train_test_split(X,y, train_size=0.7, random_state=42, shuffle=True)

modelo_xgb = XGBRegressor(random_state=42,  eval_metric='rmse')

# Definimos un espacio de parámetros para optimizar
params_xgb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 4],  # Equivalente a min_samples_leaf
    'subsample': [0.8, 1.0],        # Para evitar sobreajuste
    'colsample_bytree': [0.8, 1.0]  # Selección de columnas al construir cada árbol
}

# Optimización con GridSearchCV
random_search_xgb = GridSearchCV(modelo_xgb, 
                                 params_xgb, 
                                 cv=3, 
                                 scoring="neg_mean_squared_error", 
                                 n_jobs=-1)

random_search_xgb.fit(X_train, y_train.to_numpy().ravel())

# Mejor modelo obtenido
modelo_final_xgb = random_search_xgb.best_estimator_


y_pred_test_xg = modelo_final_xgb.predict(X_test)
y_pred_train_xg = modelo_final_xgb.predict(X_train)

resultados_xg= sacar_metricas(y_train, y_test, y_pred_train_xg, y_pred_test_xg)
resultados_xg


,r2,MAE,MSE,RMSE
train,0.865319,1559.126386,6.057622e+06,2461.223721
test,0.763334,1852.755420,1.031355e+07,3211.471231
